In [21]:
# Display the current working directory
getwd();
# If necessary, change the path below to the directory where the data files are stored.
# "." means current directory. On Windows use a forward slash / instead of the usual \.
workingDir = "/home/jovyan/work/phd/datasets/gtex/10/random_selection";
setwd(workingDir);
# Load WGCNA package
library(WGCNA)
allowWGCNAThreads()
# The following setting is important, do not omit.
options(stringsAsFactors = FALSE);

[1] "/home/jovyan/work/phd/datasets/gtex/10/random_selection"

Loading required package: dynamicTreeCut

Loading required package: fastcluster


Attaching package: ‘fastcluster’


The following object is masked from ‘package:stats’:

    hclust





Attaching package: ‘WGCNA’


The following object is masked from ‘package:stats’:

    cor




Allowing multi-threading with up to 2 threads.


In [22]:
add_legend <- function(...) {
  opar <- par(fig=c(0, 1, 0, 1), oma=c(0, 0, 0, 0), 
    mar=c(0, 0, 0, 0), new=TRUE)
  on.exit(par(opar))
  plot(0, 0, type='n', bty='n', xaxt='n', yaxt='n')
  legend(...)
}

In [23]:
datExpr <- t(read.csv("mainTable.csv", check.names=FALSE, row.names = ""))
genes <- dimnames(datExpr)[[2]]
samples <- dimnames(datExpr)[[1]]
datExpr <- as.data.frame(apply(datExpr,2, as.numeric)) #numeric
dimnames(datExpr)[[1]] <- samples
allTraits <- read.csv("files.dat")

In [24]:
log2_1 <- function(x){
    log2(x+1)
}
datExpr <- as.data.frame(apply(datExpr,2, log2_1))
dimnames(datExpr)[[1]] <- samples

In [25]:
datExpr[1:10,1:5]

,ENSG00000271614,ENSG00000258725,ENSG00000177694,ENSG00000261501,ENSG00000239910
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GTEX-111CU-1426-SM-5GZYP,1.000000,0,1.000000,0,0
GTEX-111FC-0226-SM-5N9B8,2.321928,0,1.584963,0,0
GTEX-111YS-0006-SM-5NQBE,0.000000,0,0.000000,0,0
GTEX-111YS-0326-SM-5GZZ3,0.000000,0,0.000000,0,0
GTEX-111YS-2426-SM-5GZZQ,1.000000,0,0.000000,0,0
GTEX-1122O-0005-SM-5O99J,1.000000,0,0.000000,0,0
GTEX-1122O-0426-SM-5H12G,1.584963,0,1.000000,0,0
GTEX-1122O-0626-SM-5N9B9,0.000000,0,0.000000,0,0
GTEX-1122O-1026-SM-9WYSF,0.000000,1,0.000000,0,0


In [26]:
head(allTraits)

,SAMPID,SMATSSCR,SMCENTER,SMPTHNTS,SMRIN,SMTS,SMTSD,SMUBRID,SMTSISCH,SMTSPAX,⋯,SME1ANTI,SMSPLTRD,SMBSMMRT,SME1SNSE,SME1PCTS,SMRRNART,SME1MPRT,SMNUM5CD,SMDPMPRT,SME2PCTS
,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,<dbl>
1,GTEX-1117F-0003-SM-58Q7G,NA,B1,,NA,Blood,Whole Blood,0013756,1188,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,GTEX-1117F-0003-SM-5DWSB,NA,B1,,NA,Blood,Whole Blood,0013756,1188,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,GTEX-1117F-0003-SM-6WBT7,NA,B1,,NA,Blood,Whole Blood,0013756,1188,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,GTEX-1117F-0011-R10a-SM-AHZ7F,NA,"B1, A1",,NA,Brain,Brain - Frontal Cortex (BA9),0009834,1193,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,GTEX-1117F-0011-R10b-SM-CYKQ8,NA,"B1, A1",,7.2,Brain,Brain - Frontal Cortex (BA9),0009834,1193,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,GTEX-1117F-0226-SM-5GZZ7,0,B1,"2 pieces, ~15% vessel stroma, rep delineated",6.8,Adipose Tissue,Adipose - Subcutaneous,0002190,1214,1125,⋯,14648800,11999300,0.00315785,14669500,50.0354,0.00310538,0.99474,NA,0,50.1944


In [27]:
#labels <- c('primary_site', 'malignacy', 'disease_type', 'stage')
#labels <- c('Subtype_Selected','Subtype_Selected_Lum')
#labels <- c('cases.0.project.primary_site', 'cases.0.project.disease_type')
labels <- c('SMTS', 'SMTSD')
datTraits <- allTraits[,labels]
rownames(datTraits) <- allTraits$SAMPID #<--CHECK
datTraits <- datTraits[rownames(datExpr),]
head(datTraits)

,SMTS,SMTSD
,<chr>,<chr>
GTEX-111CU-1426-SM-5GZYP,Colon,Colon - Transverse
GTEX-111FC-0226-SM-5N9B8,Adipose Tissue,Adipose - Subcutaneous
GTEX-111YS-0006-SM-5NQBE,Blood,Whole Blood
GTEX-111YS-0326-SM-5GZZ3,Heart,Heart - Atrial Appendage
GTEX-111YS-2426-SM-5GZZQ,Adipose Tissue,Adipose - Subcutaneous
GTEX-1122O-0005-SM-5O99J,Blood,Whole Blood


In [28]:
table( dimnames(datExpr)[[1]]==datTraits$SMTS)


FALSE 
 1000 

In [29]:
y = datTraits$Subtype_Selected

In [30]:
meanExpressionBySample=apply( datExpr,1,mean, na.rm=T)
NumberMissingBySample=apply( is.na(data.frame(datExpr)),1, sum)

## 1

In [31]:
gsg = goodSamplesGenes(datExpr, verbose = 3);
gsg$allOK

 Flagging genes and samples with too many missing values...
  ..step 1
  ..Excluding 888 genes from the calculation due to too many missing samples or zero variance.
  ..step 2


[1] FALSE

In [32]:
sampleTree = hclust(dist(datExpr), method = "average");
# Plot the sample tree: Open a graphic output window of size 12 by 9 inches
# The user should change the dimensions if the window is too large or too small.
sizeGrWindow(12,9)
svg("samplehier.svg")
par(cex = 0.6);
par(mar = c(0,4,2,0))
plot(sampleTree, main = "Sample clustering to detect outliers", sub="", xlab="", cex.lab = 1.5,
cex.axis = 1.5, cex.main = 2)
dev.off()

png 
  2

In [33]:
# Determine cluster under the line
clust = cutreeStatic(sampleTree, cutHeight = 1e5, minSize = 10)
table(clust)

clust
   1 
1000 

In [34]:
# clust 1 contains the samples we want to keep.
keepSamples = (clust==1)
datExpr = datExpr[keepSamples, ]
nGenes = ncol(datExpr)
nSamples = nrow(datExpr)

genes <- dimnames(datExpr)[[2]]
samples <- dimnames(datExpr)[[1]]
datExpr <- as.data.frame(apply(datExpr,2, as.numeric)) #numeric
dimnames(datExpr)[[1]] <- samples

In [35]:
head(datExpr)

,ENSG00000271614,ENSG00000258725,ENSG00000177694,ENSG00000261501,ENSG00000239910,ENSG00000263974,ENSG00000248559,ENSG00000279489,ENSG00000135407,ENSG00000170165,⋯,ENSG00000234564,ENSG00000199960,ENSG00000261570,ENSG00000223111,ENSG00000274762,ENSG00000264791,ENSG00000199787,ENSG00000118855,ENSG00000200087,ENSG00000253951
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GTEX-111CU-1426-SM-5GZYP,1.000000,0,1.000000,0,0,0,0,0,2.321928,0,⋯,0,0,0,0,0,0,0,5.392317,2.584963,0
GTEX-111FC-0226-SM-5N9B8,2.321928,0,1.584963,0,0,0,0,1,2.321928,0,⋯,0,0,0,0,0,0,0,5.643856,1.584963,0
GTEX-111YS-0006-SM-5NQBE,0.000000,0,0.000000,0,0,0,0,0,2.807355,0,⋯,0,0,0,0,0,0,0,6.392317,0.000000,0
GTEX-111YS-0326-SM-5GZZ3,0.000000,0,0.000000,0,0,0,0,0,0.000000,0,⋯,0,0,0,0,0,0,0,4.392317,0.000000,0
GTEX-111YS-2426-SM-5GZZQ,1.000000,0,0.000000,0,0,0,0,0,1.000000,0,⋯,0,0,0,0,0,0,0,5.491853,1.000000,0
GTEX-1122O-0005-SM-5O99J,1.000000,0,0.000000,0,0,0,0,0,4.392317,0,⋯,0,0,0,0,0,0,0,7.169925,2.584963,0


In [36]:
rownames(allTraits) <- allTraits$SAMPID
datTraits <- allTraits[keepSamples,labels]
rownames(datTraits) <- rownames(allTraits[keepSamples,labels])
datTraits <- datTraits[rownames(datExpr),]
head(datTraits)

,SMTS,SMTSD
,<chr>,<chr>
GTEX-111CU-1426-SM-5GZYP,Colon,Colon - Transverse
GTEX-111FC-0226-SM-5N9B8,Adipose Tissue,Adipose - Subcutaneous
GTEX-111YS-0006-SM-5NQBE,Blood,Whole Blood
GTEX-111YS-0326-SM-5GZZ3,Heart,Heart - Atrial Appendage
GTEX-111YS-2426-SM-5GZZQ,Adipose Tissue,Adipose - Subcutaneous
GTEX-1122O-0005-SM-5O99J,Blood,Whole Blood


In [37]:
collectGarbage();

In [38]:
# Re-cluster samples
sampleTree2 = hclust(dist(datExpr), method = "average")
fontsize=.5
# Convert traits to a color representation: white means low, red means high, grey means missing entry
traitColors = labels2colors(datTraits);
# Plot the sample dendrogram and the colors underneath.
pdf("samplehiercut.pdf")
plotDendroAndColors(sampleTree2, traitColors,
    groupLabels = labels,
    main = "Sample dendrogram and trait heatmap",
    dendroLabels=FALSE,
    rowText=datTraits,
    rowTextIgnore=labels[[2]]
    )
dev.off()

png 
  2

## 2

In [39]:
# Choose a set of soft-thresholding powers
powers = c(c(1:10), seq(from = 12, to=50, by=2))
# Call the network topology analysis function
sft = pickSoftThreshold(datExpr, powerVector = powers, verbose = 5)
# Plot the results:
sizeGrWindow(9, 5)
pdf("scaletopology.pdf")
par(mfrow = c(1,2));
cex1 = 0.9;
# Scale-free topology fit index as a function of the soft-thresholding power
plot(sft$fitIndices[,1], -sign(sft$fitIndices[,3])*sft$fitIndices[,2],
xlab="Soft Threshold (power)",ylab="Scale Free Topology Model Fit,signed R^2",type="n",
main = paste("Scale independence"));
text(sft$fitIndices[,1], -sign(sft$fitIndices[,3])*sft$fitIndices[,2],
labels=powers,cex=cex1,col="red");
# this line corresponds to using an R^2 cut-off of h
abline(h=0.90,col="red")
# Mean connectivity as a function of the soft-thresholding power
plot(sft$fitIndices[,1], sft$fitIndices[,5],
xlab="Soft Threshold (power)",ylab="Mean Connectivity", type="n",
main = paste("Mean connectivity"))
text(sft$fitIndices[,1], sft$fitIndices[,5], labels=powers, cex=cex1,col="red")
dev.off()

pickSoftThreshold: will use block size 3000.
 pickSoftThreshold: calculating connectivity for given powers...
   ..working on genes 1 through 3000 of 3000


Warning message in eval(xpr, envir = envir):
“Some correlations are NA in block 1 : 3000 .”
Warning message in eval(xpr, envir = envir):
“Some correlations are NA in block 1 : 3000 .”


   Power SFT.R.sq  slope truncated.R.sq mean.k. median.k. max.k.
1      1    0.868 -0.494          0.843 200.000  1.35e+02  645.0
2      2    0.942 -0.775          0.938  70.400  2.13e+01  355.0
3      3    0.957 -0.905          0.967  34.500  5.54e+00  231.0
4      4    0.925 -1.030          0.947  19.800  1.85e+00  167.0
5      5    0.916 -1.110          0.953  12.500  8.40e-01  127.0
6      6    0.906 -1.180          0.960   8.340  3.55e-01   99.1
7      7    0.902 -1.230          0.962   5.830  1.67e-01   79.2
8      8    0.900 -1.270          0.968   4.230  8.13e-02   64.3
9      9    0.904 -1.300          0.973   3.160  4.45e-02   52.9
10    10    0.886 -1.340          0.946   2.420  2.46e-02   44.0
11    12    0.864 -1.350          0.903   1.520  6.21e-03   31.3
12    14    0.884 -1.290          0.855   1.020  1.76e-03   23.8
13    16    0.325 -2.200          0.233   0.737  4.94e-04   23.5
14    18    0.361 -3.320          0.267   0.563  1.49e-04   23.2
15    20    0.395 -3.600 

png 
  2

In [40]:
net = blockwiseModules(datExpr, power = 30,
TOMType = "unsigned", minModuleSize = 5,
reassignThreshold = 0, mergeCutHeight = 0.2,
numericLabels = TRUE, pamRespectsDendro = FALSE,
saveTOMs = TRUE,
saveTOMFileBase = "wgcna",
verbose = 3)

 Calculating module eigengenes block-wise from all genes
   Flagging genes and samples with too many missing values...
    ..step 1
  ..Excluding 888 genes from the calculation due to too many missing samples or zero variance.
    ..step 2
 ..Working on block 1 .
    TOM calculation: adjacency..
    ..will use 2 parallel threads.
     Fraction of slow calculations: 0.000000
    ..connectivity..
    ..matrix multiplication (system BLAS)..
    ..normalization..
    ..done.
   ..saving TOM for block 1 into file wgcna-block.1.RData
 ....clustering..
 ....detecting modules..
 ....calculating module eigengenes..
 ....checking kME in modules..
 ..merging modules that are too close..
     mergeCloseModules: Merging modules whose distance is less than 0.2
       Calculating new MEs...


In [41]:
# open a graphics window
sizeGrWindow(12, 9)
pdf("netdendograms.pdf")
# Convert labels to colors for plotting
mergedColors = labels2colors(net$colors)
# Plot the dendrogram and the module colors underneath
plotDendroAndColors(net$dendrograms[[1]], mergedColors[net$blockGenes[[1]]],
"Module colors",
dendroLabels = FALSE, hang = 0.03,
addGuide = TRUE, guideHang = 0.05)
dev.off()

png 
  2

In [42]:
moduleLabels = net$colors
moduleColors = labels2colors(net$colors)
MEs = net$MEs;
geneTree = net$dendrograms[[1]];

## 3

In [43]:
#datTraits$cases.0.project.primary_site <- match(datTraits$cases.0.project.primary_site, unique(datTraits$cases.0.project.primary_site))
#datTraits$cases.0.project.disease_type <- match(datTraits$cases.0.project.disease_type, unique(datTraits$cases.0.project.disease_type))

ERROR: Error in `$<-.data.frame`(`*tmp*`, cases.0.project.primary_site, value = integer(0)): replacement has 0 rows, data has 1000


In [44]:
datTraits$SMTS <- match(datTraits$SMTS, unique(datTraits$SMTS))
datTraits$SMTSD <- match(datTraits$SMTSD, unique(datTraits$SMTSD))

In [ ]:
#datTraits$Subtype_Selected <- match(datTraits$Subtype_Selected, unique(datTraits$Subtype_Selected))

In [45]:
# Define numbers of genes and samples
nGenes = ncol(datExpr);
nSamples = nrow(datExpr);
# Recalculate MEs with color labels
MEs0 = moduleEigengenes(datExpr, moduleColors)$eigengenes
MEs = orderMEs(MEs0)
moduleTraitCor = cor(MEs, datTraits, use = "p");
moduleTraitPvalue = corPvalueStudent(moduleTraitCor, nSamples);

In [46]:
pdf("moduleTrait.pdf", width=9, height=18)
# Will display correlations and their p-values
textMatrix = paste(signif(moduleTraitCor, 2), "\n(",
signif(moduleTraitPvalue, 1), ")", sep = "");
dim(textMatrix) = dim(moduleTraitCor)
par(mar = c(6, 8.5, 3, 3));
# Display the correlation values within a heatmap plot
labeledHeatmap(Matrix = moduleTraitCor,
xLabels = labels,
yLabels = names(MEs),
ySymbols = names(MEs),
colorLabels = FALSE,
colors = blueWhiteRed(50),
textMatrix = textMatrix,
setStdMargins = FALSE,
cex.text = 0.5,
zlim = c(-1,1),
main = paste("Module-trait relationships"))
dev.off()

png 
  2

In [47]:
# Define variable weight containing the weight column of datTrait
weight = as.data.frame(rownames(datTraits));
names(weight) = "weight"
# names (colors) of the modules
modNames = substring(names(MEs), 3)
geneModuleMembership = as.data.frame(cor(datExpr, MEs, use = "p"));
MMPvalue = as.data.frame(corPvalueStudent(as.matrix(geneModuleMembership), nSamples));
names(geneModuleMembership) = paste("MM", modNames, sep="");
names(MMPvalue) = paste("p.MM", modNames, sep="");
geneTraitSignificance = as.data.frame(cor(datExpr, weight, use = "p"));
GSPvalue = as.data.frame(corPvalueStudent(as.matrix(geneTraitSignificance), nSamples));
names(geneTraitSignificance) = paste("GS.", names(weight), sep="");
names(GSPvalue) = paste("p.GS.", names(weight), sep="");

Warning message in storage.mode(y) <- "double":
“NAs introduced by coercion”


In [ ]:
module = "gray"
column = match(module, modNames);
moduleGenes = moduleColors==module;
sizeGrWindow(7, 7);
pdf("modulemembership.pdf")
par(mfrow = c(1,1));
verboseScatterplot(abs(geneModuleMembership[moduleGenes, column]),
    abs(geneTraitSignificance[moduleGenes, 1]),
    xlab = paste("Module Membership in", module, "module"),
    ylab = "Gene significance for primary_site",
    main = paste("Module membership vs. gene significance\n"),
    cex.main = 1.2, cex.lab = 1.2, cex.axis = 1.2, col = module)
dev.off()

In [ ]:
# Create the starting data frame
geneInfo0 = data.frame(
moduleColor = moduleColors,
geneTraitSignificance,
GSPvalue)
# Order modules by their significance for weight
modOrder = order(-abs(cor(MEs, weight, use = "p")));
# Add module membership information in the chosen order
for (mod in 1:ncol(geneModuleMembership))
{
oldNames = names(geneInfo0)
geneInfo0 = data.frame(geneInfo0, geneModuleMembership[, modOrder[mod]],
MMPvalue[, modOrder[mod]]);
names(geneInfo0) = c(oldNames, paste("MM.", modNames[modOrder[mod]], sep=""),
paste("p.MM.", modNames[modOrder[mod]], sep=""))
}
# Order the genes in the geneInfo variable first by module color, then by geneTraitSignificance
geneOrder = order(geneInfo0$moduleColor, -abs(geneInfo0$GS.weight));
geneInfo = geneInfo0[geneOrder, ]
write.csv(geneInfo, file = "geneInfo.csv")

In [ ]:
# Read in the probe annotation
annot = read.csv(file = "geneInfo.csv");
# Match probes in the data set to the probe IDs in the annotation file
probes = names(datExpr)
allLLIDs = probes
# $ Choose interesting modules
intModules = c("greenyellow", "darkorange", "darkgreen")
for (module in intModules)
{
# Select module probes
modGenes = (moduleColors==module)
# Get their entrez ID codes
modLLIDs = probes[modGenes];
# Write them into a file
fileName = paste("LocusLinkIDs-", module, ".txt", sep="");
write.table(as.data.frame(modLLIDs), file = fileName,
row.names = FALSE, col.names = FALSE, quote=FALSE)
}
# As background in the enrichment analysis, we will use all probes in the analysis.
fileName = paste("LocusLinkIDs-all.txt", sep="");
write.table(as.data.frame(allLLIDs), file = fileName,
row.names = FALSE, col.names = FALSE, quote=FALSE)

In [ ]:
require("biomaRt")
mart <- useMart("ENSEMBL_MART_ENSEMBL")
mart <- useDataset("hsapiens_gene_ensembl", mart)

ens <- allLLIDs
ensLookup <- gsub("\\.[0-9]*$", "", ens)

annotLookup <- getBM(
  mart=mart,
  attributes=c("ensembl_transcript_id", "ensembl_gene_id", "gene_biotype", "external_gene_name"),
  filter="ensembl_gene_id",
  values=ensLookup,
  uniqueRows=TRUE)

annotLookup <- data.frame(
  ens[match(annotLookup$ensembl_gene_id, ensLookup)],
  annotLookup)

colnames(annotLookup) <- c(
  "original_id",
  c("ensembl_transcript_id", "ensembl_gene_id", "gene_biotype", "external_gene_name"))

annotLookup

## 5 

In [48]:
# Re-cluster samples
sampleTree3 = hclust(dist(MEs0), method = "average")
# Convert traits to a color representation: white means low, red means high, grey means missing entry
traitColors = labels2colors(datTraits);
# Plot the sample dendrogram and the colors underneath.
pdf("nethier.pdf", width=10)
# Set the plot margin: bottom, left, top & right
par(mar = c(5, 4, 1.4, 0.2) + 0.1,
    xpd = NA) # allow content to go into outer margin 
plotDendroAndColors(sampleTree3, traitColors,
groupLabels = labels,
main = "Network dendrogram and trait heatmap",
dendroLabels=FALSE,
#rowText=allTraits[keepSamples,labels],
#rowTextIgnore=labels[[2]],
addGuide=TRUE)
dev.off()

png 
  2

In [ ]:
rownames(allTraits)=allTraits$file_name
unique(allTraits[rownames(datTraits),'Subtype_Selected'])

In [ ]:
svg("nethier_legend.svg", width=10)
add_legend("topleft", legend = unique(allTraits[rownames(datTraits),'SMTS']), pch = 1, pt.cex = 1, cex = 1.5, bty = 'n', ncol=2,
       inset = c(0.1, 0.), # place outside
       title = "SMTS", 
       col = traitColors)
dev.off()

In [ ]:
collectGarbage()

In [ ]:
clust = cutreeStatic(sampleTree3, cutHeight = 0.4, minSize = 10)
table(clust)

In [ ]:
# Re-cluster samples after cut
sampleTree4 = hclust(dist(MEs0[(clust==1),]), method = "average")
# Convert traits to a color representation: white means low, red means high, grey means missing entry
traitColors = labels2colors(datTraits[(clust==1),]);
# Plot the sample dendrogram and the colors underneath.
svg("nethiercut.svg")
plotDendroAndColors(sampleTree4, traitColors,
groupLabels = labels,
main = "Network dendrogram and trait heatmap",
dendroLabels=FALSE)
dev.off()

In [ ]:
# Calculate topological overlap anew: this could be done more efficiently by saving the TOM
# calculated during module detection, but let us do it again here.
dissTOM = 1-TOMsimilarityFromExpr(datExpr, power = 6);
# Transform dissTOM with a power to make moderately strong connections more visible in the heatmap
plotTOM = dissTOM^7;
# Set diagonal to NA for a nicer plot
diag(plotTOM) = NA;
# Call the plot function
sizeGrWindow(9,9)
svg("networkmap.svg")
TOMplot(plotTOM, geneTree, moduleColors, main = "Network heatmap plot, all genes")
dev.off()

In [ ]:
#selct genes to be faster
nSelect = nGenes*0.7
# For reproducibility, we set the random seed
set.seed(10);
select = sample(nGenes, size = nSelect);
selectTOM = dissTOM[select, select];
# There’s no simple way of restricting a clustering tree to a subset of genes, so we must re-cluster.
selectTree = hclust(as.dist(selectTOM), method = "average")
selectColors = moduleColors[select];
# Open a graphical window
sizeGrWindow(9,9)
svg("networkmap.svg")
# Taking the dissimilarity to a power, say 10, makes the plot more informative by effectively changing
# the color palette; setting the diagonal to NA also improves the clarity of the plot
plotDiss = selectTOM^7;
diag(plotDiss) = NA;
TOMplot(plotDiss, selectTree, selectColors, main = "Network heatmap plot, selected genes")
dev.off()

In [ ]:
# Recalculate module eigengenes
MEs = moduleEigengenes(datExpr, moduleColors)$eigengenes
# Isolate weight from the clinical traits
weight = as.data.frame(datTraits$SMTS);
names(weight) = "SMTS"
# Add the weight to existing module eigengenes
MET = orderMEs(cbind(MEs, weight))
# Plot the relationships among the eigengenes and the trait
sizeGrWindow(5,7.5);
svg("eigenHeat.svg")
par(cex = 0.9)
plotEigengeneNetworks(MET, "", marDendro = c(0,4,1,2), marHeatmap = c(3,4,1,2), cex.lab = 0.8, xLabelsAngle = 90)
dev.off()

In [ ]:
# Plot the dendrogram
sizeGrWindow(6,6);
svg("eigendendogram.svg")
par(cex = 1.0)
plotEigengeneNetworks(MET, "Eigengene dendrogram", marDendro = c(0,4,2,0),
plotHeatmaps = FALSE)
# Plot the heatmap matrix (note: this plot will overwrite the dendrogram plot)
par(cex = 1.0)
plotEigengeneNetworks(MET, "Eigengene adjacency heatmap", marHeatmap = c(3,4,2,2),
plotDendrograms = FALSE, xLabelsAngle = 90)
dev.off()

In [50]:
# [227  31  10   2   1]
write.csv(cutree(sampleTree3, k=1), "wgcna/wgcna_level_4_labels.csv")
write.csv(cutree(sampleTree3, k=2), "wgcna/wgcna_level_3_labels.csv")
write.csv(cutree(sampleTree3, k=10), "wgcna/wgcna_level_2_labels.csv")
write.csv(cutree(sampleTree3, k=31), "wgcna/wgcna_level_1_labels.csv")
write.csv(cutree(sampleTree3, k=227), "wgcna/wgcna_level_0_labels.csv")

In [51]:
write.csv(geneModuleMembership, "wgcna/wgcna_level_0_word-dist.csv")

In [52]:
write.csv(MEs, "wgcna/wgcna_level_0_topic-dist.csv")